## CNN 

In [9]:
from PyFunctions.Functions import *
from PyFunctions import var
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras import regularizers
import os
from tqdm import tqdm

In [3]:
def get_conv_model_normal(dim):
    '''This function will compile and return a CNN neural network given the input dimension'''
    inp_shape = dim
    act = 'relu'
    drop = .25
    kernal_reg = regularizers.l1(.001)
    optimizer = Adam(lr = .0001)
    
    model = Sequential() 
    
    model.add(Conv2D(64, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = 'he_uniform',  padding = 'same', name = 'Input_Layer'))
#     model.add(Dense(64, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    
    
    model.add(Conv2D(64, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
#     model.add(Dense(64, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))
    

    
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))

    
    model.add(Flatten())

    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))

    model.add(Dropout(drop))

    model.add(Dense(3, activation='softmax', name = 'Output_Layer'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model 




### Images

- Within the folder tests, add whatever image you would like that contains a gun and run the cell below

In [4]:
edge= True 
if edge == True: 
    dim = (var.norm_dimension[0],var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeightsNoAugment.h5') #path to the model weights

test_folder = 'Tests/Photos'

for i in os.listdir(test_folder):
    if i == 'ipynb_checkpoints': 
        continue
    img_path = f'{test_folder}/{i}'
    predictions = get_img_prediction_bounding_box(img_path, normal_model, dim = var.norm_dimension, edge = edge)

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Creating Bounding Boxes for Tests/Photos/AR.jpg
Creating Bounding Boxes for Tests/Photos/Pistol.jpg
Creating Bounding Boxes for Tests/Photos/Pistol3.jpg


### Video 
- within the folder Tests, add whatever video (.mp4) that contains a gun and run the cell below
- **Note**: Stick t video shorter than a minute or this process will take a very long time depending on your computer

In [7]:
edge= True 
if edge == True: 
    dim = (var.norm_dimension[0],var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('ModelWeights/CNN-ModelCheckpointWeightsNoAugment.h5')

images = get_vid_frames('../Tests/Videos/Pistol2.mp4', normal_model, var.norm_dimension, edge = True)

Splitting Video Into 720 Frames: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 720/720 [00:02<00:00, 323.61it/s]


In [10]:
pbar = tqdm(images, desc= 'Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While')
windows_prob = [window_prob_func(img, normal_model, var.norm_dimension, edge = edge) for img in pbar]

Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While: 100%|██████████████████████████████████████████████████████| 720/720 [04:03<00:00,  2.95it/s]


In [11]:
vid_dim = (224,224)
vid_dim = var.norm_dimension
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Tests/VideoOutput/BoundingBox.mp4',fourcc, 10, vid_dim) #change the filename to whatever you would like


for prob, img in zip(windows_prob, images): 
    clone = img.copy()
    if prob == None:
        out.write(clone)
    else:
        (p, (startx, starty, endx, endy)) = prob
        cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
        out.write(clone)
        
out.release()
cv2.destroyAllWindows()